<p><img alt="UNQ logo" height="45px" src="http://www.unq.edu.ar/images/logo_unqui_footer.png" align="left" hspace="10px" vspace="0px"></p><h1>Redes Neuronales y Lógica Difusa</h1>

# Métodos de optimización iterativos basados en el gradiente descendente

Importo módulos a utilizar

In [0]:
import numpy as np
import matplotlib.pyplot as plt

Defino el espacio de trabajo

In [0]:
xlim = [-2, 2]
dx = 0.1
ylim = [-2, 2]
dy = 0.1
xpts = np.arange(xlim[0], xlim[1], dx)
ypts = np.arange(ylim[0], ylim[1], dy)

Armo el meshgrid

In [0]:
[X, Y] = np.meshgrid(xpts, ypts)

Defino coeficientes de la cuadrática

In [0]:
a = np.array([[6, - 2], [-2, 6]])
b = np.array([-1, 1])
c = 0

Defino las constantes

In [0]:
lr = 0.1
lr_min = 0.01
lr_max = 0.25 # Los autovalores son 4 y 8. lr_max = 2 / aut_max = 0.25
F = (a[0][0] * X**2 + (a[0][1] + a[1][0]) * X * Y + a[1][1] * Y**2) / 2 + \
    b[0] * X + b[1] * Y + c

Defino las condiciones iniciales

In [0]:
x = 1
y = 0.5
x1i = np.array([x, y])
vx1 = np.array([x1i])
vx2 = x1i
vx3 = x1i

Calculo el error inicial

In [0]:
F1 = 0.5 * vx1[0].transpose() * a * vx1[0] + b * vx1[0] + c

Implemento el agoritmo de gradiente descendiente y grafico

In [0]:
f1 = plt.figure(1)
plt.contour(xpts, ypts, F)

vF = []

for i in range(100):
    vF.append(0.5 * vx1[i,:].transpose() @ a @ vx1[i,:] + b @ vx1[i,:] + c)
    grad = a @ vx1[i,:] + b.transpose()
    p = -grad
    dx1 = -lr * grad
    vx1 = np.concatenate((vx1, [vx1[i,:] + dx1]), axis=0)

plt.plot(vx1[:,0], vx1[:,1], 'ko-')
plt.title('Método gradiente descendente')

f1.show()

f2 = plt.figure(2)
plt.plot(abs(np.diff(vF)), 'ko-')
plt.title('Error de gradiente descendente')
f2.show()

# Minimización de funciones

Importo módulos a utilizar. En este caso además de los que venimos utilizando vamos a utilizar también el módulo [*sympy*](https://www.sympy.org/en/index.html), el cual nos brinda métodos para trabajar con matemáticas simbólicas.

In [0]:
import sympy
import numpy as np
import matplotlib.pyplot as plt

Defino mi función para el cálculo del gradiente. Para hacer esto haremos uso de la función [*lambda*](https://www.w3schools.com/python/python_lambda.asp).

In [0]:
fGradiente = lambda f, v: sympy.Matrix([f]).jacobian(v)

Defino el espacio de trabajo

In [0]:
xlim = [-2, 2]
dx = 0.1
ylim = [-2, 2]
dy = 0.1
xpts = np.arange(xlim[0], xlim[1], dx)
ypts = np.arange(ylim[0], ylim[1], dy)

Armo el meshgrid

In [0]:
[X1, X2] = np.meshgrid(xpts, ypts)

Defino las constantes

In [0]:
lr = 0.01
lr_min = 0.01
lr_max = 0.2  # lr_max = 2 / aut_max
nep = 14

Defino las condiciones iniciales

In [0]:
xIni = (2 * np.random.rand()) - 1
yIni = (2 * np.random.rand()) - 1

Defino las variables y funciones simbólicas

In [0]:
x1, x2 = sympy.symbols('x1 x2')
F = (X2 - 2 * X1 + np.exp(-X1))**2 + (X1 - 2 * X2 + np.exp(-X2))**2
Fsyms = (x2 - 2 * x1 + sympy.exp(-x1))**2 + (x1 - 2 * x2 + sympy.exp(-x2))**2
var = list(sympy.ordered(Fsyms.free_symbols))

gradiente = fGradiente(Fsyms, var)
hessiano = sympy.hessian(Fsyms, var)

## Método de gradiente descendente

Inicializo los vectores

In [0]:
vx1 = np.zeros((2, nep+1))
vx1[0,0] = xIni
vx1[1,0] = yIni

Calculo el error inicial

In [0]:
error_gradiente = np.zeros((nep,1))

Implemento el algoritmo de gradiente descendente y grafico

In [0]:
f1 = plt.figure(1)
ax = f1.gca(projection='3d')
ax.plot_surface(X1, X2, F, cmap='viridis', edgecolor='none')

for i in range(nep):
    error_gradiente[i] = Fsyms.subs([(x1, vx1[0, i]), (x2, vx1[1, i])])
    grad = gradiente.subs([(x1, vx1[0,i]), (x2, vx1[1, i])])
    p = -np.asarray(grad)[0]
    dx1 = lr * p
    vx1[:,i+1] = vx1[:,i] + dx1[:]

x = vx1[0, :]
y = vx1[1, :]

#ax.plot3D(x, y, Fsyms.subs([(x1, x), (x2, y)]))
f1.show()
f2 = plt.figure(2)
plt.contour(xpts, ypts, F)
plt.plot(x, y, 'ko-', linewidth=1, markersize=3)
plt.title('Método gradiente descendente')
f2.show()

## Método steepest descent 

Inicializo los vectores

In [0]:
vx2 = np.zeros((2, nep+1))
vx2[0,0] = xIni
vx2[1,0] = yIni

Calculo el error inicial

In [0]:
error_steppest = np.zeros((nep,1))

Implemento el algoritmo de steepest descent y grafico

In [0]:
for i in range(nep):
    error_steppest[i] = Fsyms.subs([(x1, vx2[0, i]), (x2, vx2[1, i])])
    grad = gradiente.subs([(x1, vx2[0, i]), (x2, vx2[1, i])]).transpose()
    p = -grad
    hess = hessiano.subs([(x1, vx2[0, i]), (x2, vx2[1, i])])
    lr = -(grad.transpose() @ p)[0] / (p.transpose() @ hess @ p)[0]
    dx2 = -lr * grad
    vx2[:,i+1] = vx2[:,i] + dx2[:]

x = vx2[0, :]
y = vx2[1, :]
#ax.plot3D(x, y, Fsyms.subs([(x1, x), (x2, y)]))

f3 = plt.figure(4)
plt.contour(xpts, ypts, F)
plt.plot(x, y, 'ko-', linewidth=1, markersize=3)
plt.title('Método steepest descent')
f3.show()

Comparo los errores entre ambos métodos

In [0]:
f4 = plt.figure(5)
plt.plot(error_gradiente, '-k', label = 'Error gradiente')
plt.plot(error_steppest, '-b', label = 'Error steppest')
f4.legend(loc = 'upper right')
plt.title('Comparación de errores')
f4.show()

# Referencias útiles

Adjunto algunos links útiles a la hora de migrar aplicaciones de MATLAB a Python que resultan muy útiles para quienes 

https://numpy.org/doc/stable/user/numpy-for-matlab-users.html?highlight=numpy%20matlab%20users

http://mathesaurus.sourceforge.net/matlab-numpy.html

https://github.com/sympy/sympy/wiki/SymPy-vs.-Matlab